# Shared protein-coding neoantigens

This script may need to be adapted, since it is done considering only 4 datasets

In [ ]:
import os
import pandas as pd
import glob
import numpy as np
from Bio import SeqIO

DATADIR="data"
GENERAL = DATADIR

projects = [f.name for f in os.scandir(DATADIR) if f.is_dir()]
sep_projects = "_".join(projects)

###if projects names are not the GEO ids, we suggest to change them. Keys correspond to project names as they are in the folder and values the GEO id
#dict_projects = {'liver_adjacent_totalRNA':'GSE101432', 'hcc_normal_totalRNA':'GSE77314','zou_hcc_RP_totalRNA':'GSE112705', 'GSE193567':'GSE193567'}
#GEO = dict_projects.values()
#GEO_list = list(GEO)

In [ ]:
A_neoantigens = pd.read_csv(os.path.join(DATADIR,projects[0],"/analysis/protein_coding_neoantigens.csv"),header=0)
B_neoantigens = pd.read_csv(os.path.join(DATADIR,projects[1],"/analysis/protein_coding_neoantigens.csv"),header=0)
C_neoantigens = pd.read_csv(os.path.join(DATADIR,projects[2],"/analysis/protein_coding_neoantigens.csv"),header=0)
D_neoantigens = pd.read_csv(os.path.join(DATADIR,projects[3],"/analysis/protein_coding_neoantigens.csv"),header=0)

In [ ]:
listA = A_neoantigens.peptide.values.tolist()
shared_in_B = B_neoantigens[B_neoantigens['peptide'].isin(listA)]
shared_in_B
total_shared = shared_in_B.merge(A_neoantigens, on='peptide')

listC = C_neoantigens.peptide.values.tolist()
shared_in_3 = total_shared[total_shared['peptide'].isin(listC)]
total_shared_3 = total_shared.merge(C_neoantigens, on='peptide')
total_shared_3.rename(columns = {'counts_x':'counts_B', 'counts_y':'counts_A', 'counts':'counts_C'}, inplace = True)

D_list = D_neoantigens.peptide.values.tolist()
shared_in_4 = total_shared_3[total_shared_3['peptide'].isin(D_list)]
total_shared_4 = total_shared_3.merge(D_neoantigens, on='peptide')
total_shared_4.rename(columns = {'counts':'counts_D'}, inplace = True)

total_shared_4.to_csv(DATADIR+"/"+sep_projects+"_proteincoding_neoantigens.csv", index=False)
total_shared_4.head(5)

Add indentifiers

In [ ]:
INDIR=os.path.join(DATADIR,projects[0],"analysis/11_PeptideBindingMHC/noncanonical_CIPHER")
lncRNA_neoantigens = total_shared_3.peptide.values.tolist()
match = list()
for neo in lncRNA_neoantigens:
    for patient in os.listdir(INDIR):
        with open(os.path.join(INDIR,patient)+"/"+patient+"_peptides_GTEx.csv") as inp:
            content=inp.readlines()
            for line in content:
                count = line.count(neo)
                if count > 0:
                    match.append(line)

In [ ]:
peptides_dict=dict()
for line in match:
    peptide=line.split(',')[0]
    ENST=line.split(',')[1]

    peptides_dict[peptide] = ENST
    
peptides_df = pd.DataFrame(peptides_dict.items(), columns=['peptide', 'transcript_id'])
total_shared_4=peptides_df.merge(total_shared_4, on='peptide')
total_shared_4.to_csv(DATADIR+"/"+sep_projects+"_proteincoding_neoantigens_ID.csv", index=False)
total_shared_4.head(10)

Immunoevidence for shared neoantigens

In [ ]:
chong = pd.read_excel("immunopeptidomes_evidences/Chong_etal_2020_SupData3_41467_2020_14968_MOESM5_ESM.xlsx", skiprows=1)
chong['Transcript_ID'] = chong['Transcript_ID'].str[:-2]
to_compare_chong = chong.Sequence.values.tolist()

spencer = pd.read_csv("immunopeptidomes_evidences/SPENCER_Immunogenic_peptide_info.txt", sep="\t")
spencer_to_compare = spencer.sequence.values.tolist()

lncpep = pd.read_csv("immunopeptidomes_evidences/pep.info.human.txt", sep="\t")
lncpep_to_compare = lncpep.orf_sequence.values.tolist()


In [ ]:
whole_evidences = dict()
for neo in lncRNA_neoantigens:
    evidences = list()
    for i in to_compare_chong:
        if neo in i and "Chong" not in evidences:
            evidences.append("chong")
    for e in spencer_to_compare:
         if neo in e and "SPENCER" not in evidences:
            evidences.append("SPENCER")
    for l in lncpep_to_compare:
        if neo in l and "LncPep" not in evidences:
            evidences.append("LncPep")
            

    whole_evidences[neo] = evidences

In [ ]:
new_dict=dict()
for key, value in whole_evidences.items():
    if len(value) > 1:
        listToStr = ','.join(map(str, value))
        new_dict[key] = listToStr
    elif len(value) == 1:
        listToStr = ''.join(map(str, value))
        new_dict[key] = listToStr
    else:
        new_dict[key] = "None"
novel_immuno_df = pd.DataFrame(new_dict.items(), columns=['peptide', 'source_evidence'])


In [ ]:
whole_df = total_shared_4.merge(novel_immuno_df, on="peptide")
whole_df.to_csv(DATADIR+"/"+sep_projects+"_proteincoding_neoantigens_immunoevidence.csv", index=None)
whole_df_not_none = whole_df[whole_df['source_evidence'] != "None"]
whole_df_not_none